In [68]:
from tweepy import Stream
import json
from keys import *
from companies import *
import pandas as pd
import time

In [142]:
class TwitterStream(Stream):

    def __init__(self, key, secret, token, token_secret, responses, limit=1):
        """Configure the SentimentListener."""
        self.responses = responses
        self.tweet_count = 0
        self.TWEET_LIMIT = limit

        super().__init__(key, secret, token, token_secret)


    """
    Inherits from tweepy's Stream class. We are only modifying the few functions
    we need to customize, namely what to do when new tweets come through.
    """

    def get_companies(self, tweet):
        """
        :param tweet: The string text of a tweet.
        :return: A list of companies mentioned in the tweet.
        """
        tweet = tweet.lower()
        companies_mentioned = []
        for stock in stocks:
            if stock in tweet:
                companies_mentioned.append(stock)

        return companies_mentioned

    def on_data(self, raw_data):
        """
        Gets called every time a new tweet gets filtered through.
        """
        self.responses = self.responses.append(self.process_data(raw_data))

        self.tweet_count += 1  # track number of tweets processed

        # if TWEET_LIMIT is reached, return False to terminate streaming
        if self.tweet_count == self.TWEET_LIMIT:
            self.disconnect()



    def process_data(self, raw_data):
        """
        Sends data through the Kafka log.
        """
        response = json.loads(raw_data)
        

        # Get tweet text
        if 'extended_tweet' in response:
            tweet_text = response['extended_tweet']['full_text']
        else:
            tweet_text = response['text']
        

        # Associate each company mentioned with the tweet
        companies_mentioned = self.get_companies(tweet_text)
        for corp in companies_mentioned:
            response['company_name'] = corp

        return response
            
    def on_error(self, status_code):
        """
        Returning false disconnects the stream.
        """
        if status_code == 420:
            return False

In [143]:
limit = 1
responses= []
stream = TwitterStream(API_KEY, API_KEY_SECERT,ACCESS_TOKEN, ACCESS_TOKEN_SECRET, responses, limit)

In [144]:
stream.filter(track=['Microsoft'], languages=['en'])


Stream connection closed by Twitter


In [145]:
print(responses)

[{'created_at': 'Wed Mar 23 02:23:04 +0000 2022', 'id': 1506456466420801536, 'id_str': '1506456466420801536', 'text': 'RT @pnjaban: Well hush my mouth...the @nytimes writes a relatively fair piece about the DOJ spying on Project Veritas through multiple Micr…', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 541220277, 'id_str': '541220277', 'name': 'LauraLee Waters', 'screen_name': 'LauraLeeWaters1', 'location': None, 'url': None, 'description': 'Christian (Catholic) wife and mother who loves God, USA, Family, Constitution, and Pro Israel. USA is a Constitutional Republic, not a Democracy!', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 315, 'friends_count': 1011, 'listed_count': 7, 'favourites_count': 3016